In [2]:
import numpy as np
import pandas as pd
import torch
import torchtext

import matplotlib.pyplot as plt

In [96]:
# load data from csv file
fields = ['news_article', 'news_category']

train_data = pd.read_csv('/content/inshort_news_data-train.csv', header=0, encoding='ISO-8859-1', usecols=fields, skip_blank_lines=True)
val_data = pd.read_csv('/content/inshort_news_data-val.csv', header=0, encoding='ISO-8859-1', usecols=fields, skip_blank_lines=True)
test_data = pd.read_csv('/content/inshort_news_data-test.csv', header=0, encoding='ISO-8859-1', usecols=fields, skip_blank_lines=True)


In [87]:
print('Num training articles: ', len(train_data))
print('Num validation articles: ', len(val_data))
print('Num testing articles: ', len(test_data))

Num training articles:  6380
Num validation articles:  1560
Num testing articles:  1742


In [97]:
# Creating training and testing data
X_train = train_data['news_article']
Y_train = train_data['news_category']
"""
Y_train = np.zeros((X_train.shape[0],1))
for i in range((X_train.shape[0])):
  for j in range(7):
    if (train[j+1][i]==1):
      Y_train[i]=j
"""
X_test = test_data['news_article']
Y_test = test_data['news_category']
"""
Y_test = np.zeros((X_test.shape[0],1))
for i in range((X_test.shape[0])):
  for j in range(7):
    if (test[j+1][i]==1):
      Y_test[i]=j
"""
X_val = val_data['news_article']
Y_val = val_data['news_category']

print (X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape)

(6380,) (6380,) (1560,) (1560,) (1742,) (1742,)


In [98]:
for i in range(X_train.shape[0]):
  X_train[i] = X_train[i].split()

for j in range(X_val.shape[0]):
  X_val[j] = X_val[j].split()

for k in range(X_test.shape[0]):
  X_test[k] = X_test[k].split()
    
Y_train = pd.get_dummies(Y_train)

In [99]:
print(X_train[0])
print(type(X_train))
print(X_train)

print(type(Y_train))
print(Y_train)

["DeepMind's", 'AI', 'system', "'AlphaFold'", 'has', 'been', 'recognised', 'as', 'a', 'solution', 'to', '"protein', 'folding",', 'a', 'grand', 'challenge', 'in', 'biology', 'for', 'over', '50', 'years.', 'DeepMind', 'showed', 'it', 'can', 'predict', 'how', 'proteins', 'fold', 'into', '3D', 'shapes,', 'a', 'complex', 'process', 'that', 'is', 'fundamental', 'to', 'understanding', 'the', 'biological', 'machinery', 'of', 'life.', 'AlphaFold', 'can', 'predict', 'the', 'shape', 'of', 'proteins', 'within', 'the', 'width', 'of', 'an', 'atom.']
<class 'pandas.core.series.Series'>
0       [DeepMind's, AI, system, 'AlphaFold', has, bee...
1       [Microsoft, Teams, will, stop, working, on, In...
2       [China,, in, response, to, reports, of, US, ad...
3       [The, global, smartphone, sales, in, the, thir...
4       [The, European, Union, (EU), is, hoping, that,...
                              ...                        
6375    [China, will, open, the, world's, largest, rad...
6376    [China's

In [101]:
# to see what is the largest number of words in a article
# np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)
# np.unique(np.array([len(ix) for ix in X_val]) , return_counts=True)
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]),
 array([  6,   5,   8,  19,  19,  71,  87,  67, 169, 178, 317, 794,   2]))

In [102]:
# stopwords to eliminate useless words
stopwords = []
stop = open('/content/stopwords.txt', encoding="utf-8")
for line in stop:
  stopwords.append(line.strip())
stop.close()

In [103]:
# utilize Glove6B for embedding
glove = torchtext.vocab.GloVe(name='6B', dim=50)

In [106]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 61, 50))
embedding_matrix_val = np.zeros((X_train.shape[0], 61, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 61, 50))

for i in range(X_train.shape[0]):
  for j in range(len(X_train[i])):
    if not (X_train[i][j].lower() in stopwords):
      embedding_matrix_train[i][j] = glove[X_train[i][j].lower()]

for i in range(X_val.shape[0]):
  for j in range(len(X_val[i])):
    if not (X_val[i][j].lower() in stopwords):
      embedding_matrix_val[i][j] = glove[X_val[i][j].lower()]

for i in range(X_test.shape[0]):
  for j in range(len(X_test[i])):
    if not (X_test[i][j].lower() in stopwords):
      embedding_matrix_test[i][j] = glove[X_test[i][j].lower()] 